In [23]:
import pandas as pd
import numpy as np
import pickle
import json
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix

In [24]:
dataset=pd.read_csv(r"C:\Users\bhara\OneDrive\Desktop\Chatbot\Medicine-Recommendation-System-Personalized-Medical-Recommendation-System-with-Machine-Learning\Training.csv")
X = dataset.drop('prognosis', axis=1)
y = dataset['prognosis']

In [25]:
le = LabelEncoder()
Y = le.fit_transform(y)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=20)


In [27]:
models = {
    'SVC': SVC(kernel='linear'),
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
    'GradientBoosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'KNeighbors': KNeighborsClassifier(n_neighbors=5),
    'MultinomialNB': MultinomialNB()
}

In [28]:
for model_name, model in models.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f"{model_name} Accuracy: {accuracy}")

SVC Accuracy: 1.0
RandomForest Accuracy: 1.0
GradientBoosting Accuracy: 1.0
KNeighbors Accuracy: 1.0
MultinomialNB Accuracy: 1.0


In [29]:
svc = models['SVC']
pickle.dump(svc, open('svc.pkl', 'wb'))

In [30]:
svc = pickle.load(open('svc.pkl', 'rb'))
sym_des=pd.read_csv(r"C:\Users\bhara\OneDrive\Desktop\Chatbot\Medicine-Recommendation-System-Personalized-Medical-Recommendation-System-with-Machine-Learning\symtoms_df.csv")
precautions = pd.read_csv(r"C:\Users\bhara\OneDrive\Desktop\Chatbot\Medicine-Recommendation-System-Personalized-Medical-Recommendation-System-with-Machine-Learning\precautions_df.csv")
workout = pd.read_csv(r"C:\Users\bhara\OneDrive\Desktop\Chatbot\Medicine-Recommendation-System-Personalized-Medical-Recommendation-System-with-Machine-Learning\workout_df.csv")
description = pd.read_csv(r"C:\Users\bhara\OneDrive\Desktop\Chatbot\Medicine-Recommendation-System-Personalized-Medical-Recommendation-System-with-Machine-Learning\description.csv")
medications = pd.read_csv(r"C:\Users\bhara\OneDrive\Desktop\Chatbot\Medicine-Recommendation-System-Personalized-Medical-Recommendation-System-with-Machine-Learning\medications.csv")
diets = pd.read_csv(r"C:\Users\bhara\OneDrive\Desktop\Chatbot\Medicine-Recommendation-System-Personalized-Medical-Recommendation-System-with-Machine-Learning\diets.csv")

In [31]:
symptoms_dict = {...}  # paste your full symptoms_dict here
diseases_list = {...}

In [83]:
def get_predicted_value(patient_symptoms):
    input_vector = np.zeros(len(symptoms_dict))
    for item in patient_symptoms:
        if item in symptoms_dict:
            input_vector[symptoms_dict[item]] = 1

    input_df = pd.DataFrame([input_vector], columns=X.columns)  # Wrap input_vector in a list
    return diseases_list[svc.predict(input_df)[0]]



In [84]:
def helper(dis):
    desc = description[description['Disease'] == dis]['Description']
    desc = " ".join(desc)

    pre = precautions[precautions['Disease'] == dis][['Precaution_1', 'Precaution_2', 'Precaution_3', 'Precaution_4']]
    pre = [col for col in pre.values]

    med = medications[medications['Disease'] == dis]['Medication']
    med = [med for med in med.values]

    die = diets[diets['Disease'] == dis]['Diet']
    die = [die for die in die.values]

    wrkout = workout[workout['disease'] == dis]['workout']

    return desc, pre, med, die, wrkout

In [85]:
USER_DATA_FILE = 'user_data.json'

In [86]:
def save_user_data(username, symptoms, prediction):
    if os.path.exists(USER_DATA_FILE):
        with open(USER_DATA_FILE, 'r') as f:
            user_data = json.load(f)
    else:
        user_data = {}

    if username not in user_data:
        user_data[username] = []

    user_data[username].append({
        'symptoms': symptoms,
        'prediction': prediction
    })

    with open(USER_DATA_FILE, 'w') as f:
        json.dump(user_data, f, indent=2)

In [87]:
def suggest_from_history(username):
    if not os.path.exists(USER_DATA_FILE):
        return []

    with open(USER_DATA_FILE, 'r') as f:
        user_data = json.load(f)

    return user_data.get(username, [])

In [88]:
username = input("Enter your username: ")

Enter your username:  xyz


In [89]:
past_records = suggest_from_history(username)
past_records

[]

In [90]:
if past_records:
    print("\n=== Your Previous Records ===")
    for record in past_records:
        print(f"Symptoms: {record['symptoms']}")
        print(f"Disease Predicted: {record['prediction']}\n")

In [91]:
symptoms = input("Enter your symptoms (comma separated): ")
user_symptoms = [s.strip().lower() for s in symptoms.split(',')]
user_symptoms = [symptom.strip("[]' ") for symptom in user_symptoms]

Enter your symptoms (comma separated):  itching,skin_rash,nodal_skin_eruptions


In [92]:
predicted_disease = get_predicted_value(user_symptoms)
desc, pre, med, die, wrkout = helper(predicted_disease)
print("\n================= Predicted Disease ================")
print(predicted_disease)
print("================= Description ======================")
print(desc)
print("================= Precautions ======================")
for i, p in enumerate(pre[0], 1):
    print(f"{i}: {p}")
print("================= Medications ======================")
for i, m in enumerate(med, 1):
    print(f"{i}: {m}")
print("================= Workout ==========================")
for i, w in enumerate(wrkout, 1):
    print(f"{i}: {w}")
print("================= Diet =============================")
for i, d in enumerate(die, 1):
    print(f"{i}: {d}")

ValueError: 132 columns passed, passed data had 1 columns